In [1]:
import datetime
import math
import os
import sys
import ee
from Py6S import *
import ee
import geemap

import geemap
import ee
from IPython.display import display

In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:
import os
import geemap
import ee

In [ ]:
import os
import geemap
import ee

def export_landsat_images(start_year, end_year, feature_collection, output_dir, cloudCover):
    ee.Initialize()

    # Define the region of interest
    jharkhand = feature_collection

    # Loop through the years
    for year in range(start_year, end_year + 1):
        images_found = False  # Flag to check if images are found for the current year

        # Create a band mapping dictionary based on the Landsat sensor
        band_mapping = {
            'LT05': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5'},
            'LE07': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5'},
            'LC08': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6'},
            'LC09': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6'}
        }

        # Get the Landsat sensor based on the year
        landsat_sensor = None
        if year < 1999:
            landsat_sensor = 'LT05'
        elif 1999 <= year < 2014:
            landsat_sensor = 'LE07'
        elif 2014 <= year < 2022:
            landsat_sensor = 'LC08'
        else:
            landsat_sensor = 'LC09'

        # Iterate through months from January to December
        for month in range(1, 13):
            # Set the start and end dates for the current month
            start_date = f'{year}-{month:02d}-01'
            end_date = f'{year}-{month:02d}-28'

            # Filter Landsat collection for the specified year and month
            landsat_collection = ee.ImageCollection(f"LANDSAT/{landsat_sensor}/C02/T1_TOA")

            images = landsat_collection.filter(ee.Filter.lt('CLOUD_COVER', cloudCover)) \
                     .filter(ee.Filter.date(start_date, end_date)) \
                     .filterBounds(jharkhand)

            # Check if images are found for the current month
            if images.size().getInfo() > 0:
                images_found = True

                # Take the mean of the images for the specified period
                image = images.mean()

                # Define output filename for Landsat
                landsat_filename = f'Landsat_{year}_{month:02d}.tif'
                landsat_output_dir = "F:/Final Images1/Landsat/"
                os.makedirs(landsat_output_dir, exist_ok=True)
                landsat_output_path = os.path.join(landsat_output_dir, landsat_filename)
                geemap.ee_export_image(image, landsat_output_path, region=jharkhand, scale=100, file_per_band=False)

                # Calculate NDVI, EVI, and NDWI
                band_info = band_mapping[landsat_sensor]
                ndvi = image.normalizedDifference([band_info['NIR'], band_info['RED']])
                evi = image.expression(
                    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                        'NIR': image.select(band_info['NIR']),
                        'RED': image.select(band_info['RED']),
                        'BLUE': image.select(band_info['BLUE'])
                    })
                ndwi = image.normalizedDifference([band_info['NIR'], band_info['SWIR']])

                # Define output filenames for NDVI, EVI, and NDWI
                ndvi_filename = f'Landsat_{year}_{month:02d}_NDVI.tif'
                evi_filename = f'Landsat_{year}_{month:02d}_EVI.tif'
                ndwi_filename = f'Landsat_{year}_{month:02d}_NDWI.tif'

                ndvi_output_dir = "F:/Final Images1/NDVI/"
                evi_output_dir = "F:/Final Images1/EVI/"
                ndwi_output_dir = "F:/Final Images1/NDWI/"

                os.makedirs(ndvi_output_dir, exist_ok=True)
                os.makedirs(evi_output_dir, exist_ok=True)
                os.makedirs(ndwi_output_dir, exist_ok=True)

                # Export NDVI, EVI, and NDWI to separate folders
                ndvi_output_path = os.path.join(ndvi_output_dir, ndvi_filename)
                evi_output_path = os.path.join(evi_output_dir, evi_filename)
                ndwi_output_path = os.path.join(ndwi_output_dir, ndwi_filename)

                geemap.ee_export_image(ndvi, ndvi_output_path, region=jharkhand, scale=30)
                geemap.ee_export_image(evi, evi_output_path, region=jharkhand, scale=30)
                geemap.ee_export_image(ndwi, ndwi_output_path, region=jharkhand, scale=30)

                break  # Exit the loop when an image is found for the current year

        if not images_found:
            print(f"No images found for year {year}. Moving to the next year.")

# Example usage:
start_year = 1984
end_year = 2022
feature_collection = ee.FeatureCollection("projects/webapp-385310/assets/Sonitpur-Udalguri").geometry()
output_dir = '/path/to/output/directory'
cloudCover = 50

# Export Landsat images, NDVI, EVI, and NDWI for each year, adding months gradually until December if necessary
export_landsat_images(start_year, end_year, feature_collection, output_dir, cloudCover)


In [ ]:
import ee
import os
import time
import geemap

def export_landsat_images(start_year, end_year, feature_collection, output_folder, cloudCover):
    ee.Initialize()

    # Define the region of interest
    jharkhand = feature_collection

    # Loop through the years
    for year in range(start_year, end_year + 1):
        images_found = False  # Flag to check if images are found for the current year

        # Create a band mapping dictionary based on the Landsat sensor
        band_mapping = {
            'LT05': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5'},
            'LE07': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5'},
            'LC08': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6'},
            'LC09': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6'}
        }

        # Get the Landsat sensor based on the year
        landsat_sensor = None
        if year < 1999:
            landsat_sensor = 'LT05'
        elif 1999 <= year < 2014:
            landsat_sensor = 'LE07'
        elif 2014 <= year < 2022:
            landsat_sensor = 'LC08'
        else:
            landsat_sensor = 'LC09'

        # Iterate through months from January to December
        for month in range(1, 13):
            # Set the start and end dates for the current month
            start_date = f'{year}-{month:02d}-01'
            end_date = f'{year}-{month:02d}-28'

            # Filter Landsat collection for the specified year and month
            landsat_collection = ee.ImageCollection(f"LANDSAT/{landsat_sensor}/C02/T1_TOA")

            images = landsat_collection.filter(ee.Filter.lt('CLOUD_COVER', cloudCover)) \
                     .filter(ee.Filter.date(start_date, end_date)) \
                     .filterBounds(jharkhand)

            # Check if images are found for the current month
            if images.size().getInfo() > 0:
                images_found = True

                # Take the mean of the images for the specified period
                image = images.mean()

                # Define output filename for Landsat
                landsat_filename = f'Landsat_{year}_{month:02d}.tif'
                landsat_output_path = os.path.join(output_folder, landsat_filename)

                # Export Landsat image to Google Drive
                export_task = ee.batch.Export.image.toDrive(
                    image=image,
                    description='Landsat Export',
                    folder=output_folder,
                    fileNamePrefix=landsat_filename,
                    scale=100,
                    region=jharkhand,
                )
                export_task.start()

                # Calculate NDVI, EVI, and NDWI
                band_info = band_mapping[landsat_sensor]
                ndvi = image.normalizedDifference([band_info['NIR'], band_info['RED']])
                evi = image.expression(
                    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                        'NIR': image.select(band_info['NIR']),
                        'RED': image.select(band_info['RED']),
                        'BLUE': image.select(band_info['BLUE'])
                    })
                ndwi = image.normalizedDifference([band_info['NIR'], band_info['SWIR']])

                # Define output filenames for NDVI, EVI, and NDWI
                ndvi_filename = f'Landsat_{year}_{month:02d}_NDVI.tif'
                evi_filename = f'Landsat_{year}_{month:02d}_EVI.tif'
                ndwi_filename = f'Landsat_{year}_{month:02d}_NDWI.tif'

                ndvi_output_path = os.path.join(output_folder, ndvi_filename)
                evi_output_path = os.path.join(output_folder, evi_filename)
                ndwi_output_path = os.path.join(output_folder, ndwi_filename)

                # Export NDVI, EVI, and NDWI to Google Drive
                export_ndvi_task = ee.batch.Export.image.toDrive(
                    image=ndvi,
                    description='NDVI Export',
                    folder=output_folder,
                    fileNamePrefix=ndvi_filename,
                    scale=30,
                    region=jharkhand,
                )
                export_evi_task = ee.batch.Export.image.toDrive(
                    image=evi,
                    description='EVI Export',
                    folder=output_folder,
                    fileNamePrefix=evi_filename,
                    scale=30,
                    region=jharkhand,
                )
                export_ndwi_task = ee.batch.Export.image.toDrive(
                    image=ndwi,
                    description='NDWI Export',
                    folder=output_folder,
                    fileNamePrefix=ndwi_filename,
                    scale=30,
                    region=jharkhand,
                )

                export_ndvi_task.start()
                export_evi_task.start()
                export_ndwi_task.start()

                # Wait for export tasks to complete
                while export_task.active() or export_ndvi_task.active() or export_evi_task.active() or export_ndwi_task.active():
                    print('Exporting...')  # You can replace this with a more informative message
                    time.sleep(30)  # Wait for 30 seconds before checking again

                print(f'Export completed for Landsat {year}_{month:02d}')

                break  # Exit the loop when an image is found for the current year

        if not images_found:
            print(f"No images found for year {year}. Moving to the next year.")

# Example usage:
start_year = 1984
end_year = 2022
feature_collection = ee.FeatureCollection("projects/webapp-385310/assets/Sonitpur-Udalguri").geometry()
output_folder = 'Earth engine'  # Replace with your Google Drive folder name
cloudCover = 50

# Export Landsat images, NDVI, EVI, and NDWI for each year, adding months gradually until December if necessary
export_landsat_images(start_year, end_year, feature_collection, output_folder, cloudCover)


In [ ]:
import ee
import os
import time
import geemap

def export_landsat_images(start_year, end_year, feature_collection, output_folder, cloudCover):
    ee.Initialize()

    # Define the region of interest
    jharkhand = feature_collection

    # Loop through the years
    for year in range(start_year, end_year + 1):
        images_found = False  # Flag to check if images are found for the current year

        # Create a band mapping dictionary based on the Landsat sensor
        band_mapping = {
            'LT05': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5'},
            'LE07': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5'},
            'LC08': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6'},
            'LC09': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6'}
        }

        # Get the Landsat sensor based on the year
        landsat_sensor = None
        if year < 1999:
            landsat_sensor = 'LT05'
        elif 1999 <= year < 2014:
            landsat_sensor = 'LE07'
        elif 2014 <= year < 2022:
            landsat_sensor = 'LC08'
        else:
            landsat_sensor = 'LC09'

        # Iterate through months from January to December
        for month in range(1, 13):
            # Set the start and end dates for the current month
            start_date = f'{year}-{month:02d}-01'
            end_date = f'{year}-{month:02d}-28'

            # Filter Landsat collection for the specified year and month
            landsat_collection = ee.ImageCollection(f"LANDSAT/{landsat_sensor}/C02/T1_TOA")

            images = landsat_collection.filter(ee.Filter.lt('CLOUD_COVER', cloudCover)) \
                     .filter(ee.Filter.date(start_date, end_date)) \
                     .filterBounds(jharkhand)

            # Check if images are found for the current month
            if images.size().getInfo() > 0:
                images_found = True

                # Take the mean of the images for the specified period
                image = images.mean()

                # Define output filename for Landsat
                landsat_filename = f'Landsat_{year}_{month:02d}.tif'
                landsat_output_path = os.path.join(output_folder, 'Landsat', landsat_filename)

#                 # Export Landsat image to Google Drive
#                 export_task = ee.batch.Export.image.toDrive(
#                     image=image,
#                     description='Landsat Export',
#                     folder=output_folder,
#                     fileNamePrefix='Landsat/' + landsat_filename,
#                     scale=30,
#                     file_per_band=True,
#                     region=jharkhand,
#                 )
#                 export_task.start()



                # Export each band of the Landsat image separately
                for band_name, band_id in band_mapping[landsat_sensor].items():
                    band_image = image.select(band_id)

                    # Define output filename for the current band
                    band_filename = f'Landsat_{year}_{month:02d}_{band_name}.tif'
                    band_output_path = os.path.join(output_folder, 'Landsat', band_filename)

                    # Export the current band to Google Drive
                    export_task = ee.batch.Export.image.toDrive(
                        image=band_image,
                        description=f'Landsat {band_name} Export',
                        folder=output_folder,
                        fileNamePrefix=f'Landsat/{band_filename}',
                        scale=30,
                        region=jharkhand,
                    )
                    export_task.start()

                    # Wait for export task to complete
                    while export_task.active():
                        print(f'Exporting {band_name}...')
                        time.sleep(30)  # Wait for 30 seconds before checking again

                print(f'Export completed for Landsat {year}_{month:02d}')



                # Calculate NDVI, EVI, and NDWI
                band_info = band_mapping[landsat_sensor]
                ndvi = image.normalizedDifference([band_info['NIR'], band_info['RED']])
                evi = image.expression(
                    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                        'NIR': image.select(band_info['NIR']),
                        'RED': image.select(band_info['RED']),
                        'BLUE': image.select(band_info['BLUE'])
                    })
                ndwi = image.normalizedDifference([band_info['NIR'], band_info['SWIR']])

                # Define output filenames for NDVI, EVI, and NDWI
                ndvi_filename = f'Landsat_{year}_{month:02d}_NDVI.tif'
                evi_filename = f'Landsat_{year}_{month:02d}_EVI.tif'
                ndwi_filename = f'Landsat_{year}_{month:02d}_NDWI.tif'

                ndvi_output_path = os.path.join(output_folder, 'NDVI', ndvi_filename)
                evi_output_path = os.path.join(output_folder, 'EVI', evi_filename)
                ndwi_output_path = os.path.join(output_folder, 'NDWI', ndwi_filename)

                # Export NDVI, EVI, and NDWI to Google Drive
                export_ndvi_task = ee.batch.Export.image.toDrive(
                    image=ndvi,
                    description='NDVI Export',
                    folder=output_folder,
                    fileNamePrefix='NDVI/' + ndvi_filename,
                    scale=30,
                    region=jharkhand,
                )
                export_evi_task = ee.batch.Export.image.toDrive(
                    image=evi,
                    description='EVI Export',
                    folder=output_folder,
                    fileNamePrefix='EVI/' + evi_filename,
                    scale=30,
                    region=jharkhand,
                )
                export_ndwi_task = ee.batch.Export.image.toDrive(
                    image=ndwi,
                    description='NDWI Export',
                    folder=output_folder,
                    fileNamePrefix='NDWI/' + ndwi_filename,
                    scale=30,
                    region=jharkhand,
                )

                export_ndvi_task.start()
                export_evi_task.start()
                export_ndwi_task.start()

                # Wait for export tasks to complete
                while export_task.active() or export_ndvi_task.active() or export_evi_task.active() or export_ndwi_task.active():
                    print('Exporting...')  # You can replace this with a more informative message
                    time.sleep(30)  # Wait for 30 seconds before checking again

                print(f'Export completed for Landsat {year}_{month:02d}')

                break  # Exit the loop when an image is found for the current year

        if not images_found:
            print(f"No images found for year {year}. Moving to the next year.")

# Example usage:
start_year = 2021
end_year = 2022
feature_collection = ee.FeatureCollection("projects/webapp-385310/assets/Sonitpur-Udalguri").geometry()
output_folder = 'GEE_Exports'  # Replace with your Google Drive folder name
cloudCover = 50

# Export Landsat images, NDVI, EVI, and NDWI for each year, adding months gradually until December if necessary
export_landsat_images(start_year, end_year, feature_collection, output_folder, cloudCover)


In [ ]:
import ee
import os
import time

def export_landsat_images(start_year, end_year, feature_collection, output_folder, cloudCover):
    ee.Initialize()

    # Define the region of interest
    jharkhand = feature_collection

    # Loop through the years
    for year in range(start_year, end_year + 1):
        images_found = False  # Flag to check if images are found for the current year

        # Create a band mapping dictionary based on the Landsat sensor
        band_mapping = {
            'LT05': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5','TIRS':'B6','GREEN':'B2'},
            'LE07': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5','TIRS':'B6_VCID_2','GREEN':'B2'},
            'LC08': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6','TIRS':'B10','GREEN':'B3'},
            'LC09': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6','TIRS':'B10','GREEN':'B3'}
        }

        # Get the Landsat sensor based on the year
        landsat_sensor = None
        if year < 1999:
            landsat_sensor = 'LT05'
        elif 1999 <= year < 2014:
            landsat_sensor = 'LE07'
        elif 2014 <= year < 2022:
            landsat_sensor = 'LC08'
        else:
            landsat_sensor = 'LC09'

        # Iterate through months from January to December
        for month in range(1, 13):
            # Set the start and end dates for the current month
            start_date = f'{year}-{month:02d}-01'
            end_date = f'{year}-{month:02d}-28'

            # Filter Landsat collection for the specified year and month
            landsat_collection = ee.ImageCollection(f"LANDSAT/{landsat_sensor}/C02/T1_TOA")

            images = landsat_collection.filter(ee.Filter.lt('CLOUD_COVER', cloudCover)) \
                     .filter(ee.Filter.date(start_date, end_date)) \
                     .filterBounds(jharkhand)

            # Check if images are found for the current month
            if images.size().getInfo() > 0:
                images_found = True

                # Take the mean of the images for the specified period
                image = images.mean()

                # Define output filename for Landsat
                landsat_filename = f'Landsat_{year}_{month:02d}.tif'
                landsat_output_path = os.path.join(output_folder, 'Landsat', landsat_filename)

                # Export each band of the Landsat image separately with export options
                for band_name, band_id in band_mapping[landsat_sensor].items():
                    band_image = image.select(band_id)

                    # Define output filename for the current band
                    band_filename = f'Landsat_{year}_{month:02d}_{band_name}.tif'
                    band_output_path = os.path.join(output_folder, 'Landsat', band_filename)

                    # Set export options for CRS and maxPixels
                    export_options = {
                        'crs': 'EPSG:32644',  # Change to your desired CRS
                        'maxPixels': 1e13  # Change to an appropriate value
                    }

                    # Export the current band to Google Drive with export_options
                    export_task = ee.batch.Export.image.toDrive(
                        image=band_image,
                        description=f'Landsat {band_name} Export',
                        folder=output_folder,
                        fileNamePrefix=f'Landsat/{band_filename}',
                        scale=30,
                        region=jharkhand,
                        **export_options  # Include export options
                    )
                    export_task.start()

                    # Wait for export task to complete
                    while export_task.active():
                        print(f'Exporting {band_name}...')
                        time.sleep(30)  # Wait for 30 seconds before checking again

                print(f'Export completed for Landsat {year}_{month:02d}')

                # Calculate NDVI, EVI, and NDWI
                band_info = band_mapping[landsat_sensor]
                ndvi = image.normalizedDifference([band_info['NIR'], band_info['RED']])
                evi = image.expression(
                    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                        'NIR': image.select(band_info['NIR']),
                        'RED': image.select(band_info['RED']),
                        'BLUE': image.select(band_info['BLUE'])
                    })
                ndwi = image.normalizedDifference([band_info['NIR'], band_info['SWIR']])

                # Define output filenames for NDVI, EVI, and NDWI
                ndvi_filename = f'Landsat_{year}_{month:02d}_NDVI.tif'
                evi_filename = f'Landsat_{year}_{month:02d}_EVI.tif'
                ndwi_filename = f'Landsat_{year}_{month:02d}_NDWI.tif'

                ndvi_output_path = os.path.join(output_folder, 'NDVI', ndvi_filename)
                evi_output_path = os.path.join(output_folder, 'EVI', evi_filename)
                ndwi_output_path = os.path.join(output_folder, 'NDWI', ndwi_filename)

                # Export NDVI, EVI, and NDWI to Google Drive with export options
                export_ndvi_task = ee.batch.Export.image.toDrive(
                    image=ndvi,
                    description='NDVI Export',
                    folder=output_folder,
                    fileNamePrefix='NDVI/' + ndvi_filename,
                    scale=30,
                    region=jharkhand,
                    **export_options  # Include export options
                )
                export_evi_task = ee.batch.Export.image.toDrive(
                    image=evi,
                    description='EVI Export',
                    folder=output_folder,
                    fileNamePrefix='EVI/' + evi_filename,
                    scale=30,
                    region=jharkhand,
                    **export_options  # Include export options
                )
                export_ndwi_task = ee.batch.Export.image.toDrive(
                    image=ndwi,
                    description='NDWI Export',
                    folder=output_folder,
                    fileNamePrefix='NDWI/' + ndwi_filename,
                    scale=30,
                    region=jharkhand,
                    **export_options  # Include export options
                )

                export_ndvi_task.start()
                export_evi_task.start()
                export_ndwi_task.start()

                # Wait for export tasks to complete
                while export_task.active() or export_ndvi_task.active() or export_evi_task.active() or export_ndwi_task.active():
                    print('Exporting...')  # You can replace this with a more informative message
                    time.sleep(30)  # Wait for 30 seconds before checking again

                print(f'Export completed for Landsat {year}_{month:02d}')

                break  # Exit the loop when an image is found for the current year

        if not images_found:
            print(f"No images found for year {year}. Moving to the next year.")

# Example usage:
start_year = 1999
end_year = 2014
feature_collection = ee.FeatureCollection("projects/webapp-385310/assets/studyarea_dl").geometry()
output_folder = 'GEE_Exports'  # Replace with your Google Drive folder name
cloudCover = 50

# Export Landsat images, NDVI, EVI, and NDWI for each year, adding months gradually until December if necessary
export_landsat_images(start_year, end_year, feature_collection, output_folder, cloudCover)


In [ ]:
import ee
import os
import time

def export_landsat_images(start_year, end_year, feature_collection, output_folder, cloudCover):
    ee.Initialize()

    # Define the region of interest
    jharkhand = feature_collection

    # Loop through the years
    for year in range(start_year, end_year + 1):
        images_found = False  # Flag to check if images are found for the current year

        # Create a band mapping dictionary based on the Landsat sensor
        band_mapping = {
            'LT05': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5','TIRS':'B6','GREEN':'B2'},
            'LE07': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5','TIRS':'B6_VCID_2','GREEN':'B2'},
            'LC08': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6','TIRS':'B10','GREEN':'B3'},
            'LC09': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6','TIRS':'B10','GREEN':'B3'}
        }

        # Get the Landsat sensor based on the year
        landsat_sensor = None
        if year < 2000:
            landsat_sensor = 'LT05'
        elif 1999 <= year < 2014:
            landsat_sensor = 'LE07'
        elif 2014 <= year < 2022:
            landsat_sensor = 'LC08'
        else:
            landsat_sensor = 'LC09'

        # Iterate through months from January to December
        for month in range(1, 13):
            # Set the start and end dates for the current month
            start_date = f'{year}-01-01'
            end_date = f'{year}-05-28'

            # Filter Landsat collection for the specified year and month
            landsat_collection = ee.ImageCollection(f"LANDSAT/{landsat_sensor}/C02/T1_TOA")

            images = landsat_collection.filter(ee.Filter.lt('CLOUD_COVER', cloudCover)) \
                     .filter(ee.Filter.date(start_date, end_date)) \
                     .filterBounds(jharkhand)

            # Check if images are found for the current month
            if images.size().getInfo() > 0:
                images_found = True

                # Take the mean of the images for the specified period
                image = images.mean()

                # Define output filename for Landsat
                landsat_filename = f'Landsat_{year}_{month:02d}.tif'
                landsat_output_path = os.path.join(output_folder, 'Landsat', landsat_filename)

                # Export each band of the Landsat image separately with export options
                for band_name, band_id in band_mapping[landsat_sensor].items():
                    band_image = image.select(band_id)

                    # Define output filename for the current band
                    band_filename = f'Landsat_{year}_{month:02d}_{band_name}.tif'
                    band_output_path = os.path.join(output_folder, 'Landsat', band_filename)

                    # Set export options for CRS and maxPixels
                    export_options = {
                        'crs': 'EPSG:32644',  # Change to your desired CRS
                        'maxPixels': 1e13  # Change to an appropriate value
                    }

                    # Export the current band to Google Drive with export_options
                    export_task = ee.batch.Export.image.toDrive(
                        image=band_image,
                        description=f'Landsat {band_name} Export',
                        folder=output_folder,
                        fileNamePrefix=f'Landsat/{band_filename}',
                        scale=30,
                        region=jharkhand,
                        **export_options  # Include export options
                    )
                    export_task.start()

                    # Wait for export task to complete
                    while export_task.active():
                        print(f'Exporting {band_name}...')
                        time.sleep(30)  # Wait for 30 seconds before checking again

                print(f'Export completed for Landsat {year}_{month:02d}')

                # Calculate NDVI, EVI, and NDWI
                band_info = band_mapping[landsat_sensor]
                ndvi = image.normalizedDifference([band_info['NIR'], band_info['RED']])
                evi = image.expression(
                    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                        'NIR': image.select(band_info['NIR']),
                        'RED': image.select(band_info['RED']),
                        'BLUE': image.select(band_info['BLUE'])
                    })
                ndwi = image.normalizedDifference([band_info['NIR'], band_info['SWIR']])

                # Define output filenames for NDVI, EVI, and NDWI
                ndvi_filename = f'Landsat_{year}_{month:02d}_NDVI.tif'
                evi_filename = f'Landsat_{year}_{month:02d}_EVI.tif'
                ndwi_filename = f'Landsat_{year}_{month:02d}_NDWI.tif'

                ndvi_output_path = os.path.join(output_folder, 'NDVI', ndvi_filename)
                evi_output_path = os.path.join(output_folder, 'EVI', evi_filename)
                ndwi_output_path = os.path.join(output_folder, 'NDWI', ndwi_filename)

                # Export NDVI, EVI, and NDWI to Google Drive with export options
                export_ndvi_task = ee.batch.Export.image.toDrive(
                    image=ndvi,
                    description='NDVI Export',
                    folder=output_folder,
                    fileNamePrefix='NDVI/' + ndvi_filename,
                    scale=30,
                    region=jharkhand,
                    **export_options  # Include export options
                )
                export_evi_task = ee.batch.Export.image.toDrive(
                    image=evi,
                    description='EVI Export',
                    folder=output_folder,
                    fileNamePrefix='EVI/' + evi_filename,
                    scale=30,
                    region=jharkhand,
                    **export_options  # Include export options
                )
                export_ndwi_task = ee.batch.Export.image.toDrive(
                    image=ndwi,
                    description='NDWI Export',
                    folder=output_folder,
                    fileNamePrefix='NDWI/' + ndwi_filename,
                    scale=30,
                    region=jharkhand,
                    **export_options  # Include export options
                )

                export_ndvi_task.start()
                export_evi_task.start()
                export_ndwi_task.start()

                # Wait for export tasks to complete
                while export_task.active() or export_ndvi_task.active() or export_evi_task.active() or export_ndwi_task.active():
                    print('Exporting...')  # You can replace this with a more informative message
                    time.sleep(30)  # Wait for 30 seconds before checking again

                print(f'Export completed for Landsat {year}_{month:02d}')

                break  # Exit the loop when an image is found for the current year

        if not images_found:
            print(f"No images found for year {year}. Moving to the next year.")

# Example usage:
start_year = 1995
end_year = 1999
feature_collection = ee.FeatureCollection("projects/webapp-385310/assets/studyarea_dl").geometry()
output_folder = 'GEE_Exports'  # Replace with your Google Drive folder name
cloudCover = 20

# Export Landsat images, NDVI, EVI, and NDWI for each year, adding months gradually until December if necessary
export_landsat_images(start_year, end_year, feature_collection, output_folder, cloudCover)


In [8]:
import ee
import os
import time

def export_landsat_images(start_year, end_year, feature_collection, output_folder, cloudCover, pathNumber, rowNumber1,rowNumber2):
    ee.Initialize()

    # Define the region of interest
    jharkhand = feature_collection

    # Loop through the years
    for year in range(start_year, end_year + 1):
        images_found = False  # Flag to check if images are found for the current year

        # Create a band mapping dictionary based on the Landsat sensor
        band_mapping = {
            'LT05': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5','TIRS':'B6','GREEN':'B2'},
            'LE07': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5','TIRS':'B6_VCID_2','GREEN':'B2'},
            'LC08': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6','TIRS':'B10','GREEN':'B3'},
            'LC09': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6','TIRS':'B10','GREEN':'B3'}
        }

        # Get the Landsat sensor based on the year
        landsat_sensor = None
        if year < 2000:
            landsat_sensor = 'LT05'
        elif 1999 <= year < 2014:
            landsat_sensor = 'LE07'
        elif 2014 <= year < 2022:
            landsat_sensor = 'LC08'
        else:
            landsat_sensor = 'LC09'

        # Iterate through months from January to December
        for month in range(1, 13):
            # Set the start and end dates for the current month
            start_date = f'{year}-01-01'
            end_date = f'{year}-12-28'

            # Filter Landsat collection for the specified year and month
            landsat_collection = ee.ImageCollection(f"LANDSAT/{landsat_sensor}/C02/T1_TOA")

            images_1 = landsat_collection.filter(ee.Filter.lt('CLOUD_COVER', cloudCover)) \
                     .filter(ee.Filter.date(start_date, end_date)) \
                     .filterBounds(jharkhand)\
                     .filter(ee.Filter.eq('WRS_ROW',rowNumber1))\
                     .filter(ee.Filter.eq('WRS_ROW',rowNumber1))\
                     .filter(ee.Filter.eq('WRS_PATH',pathNumber))\
                     .first()
            # Get the date of the first image in images_1
            first_image_date = ee.Image(images_1).date()
            

            images_2 = landsat_collection.filter(ee.Filter.lt('CLOUD_COVER', cloudCover)) \
                     .filter(ee.Filter.date(first_image_date, first_image_date.advance(1, 'day'))) \
                     .filterBounds(jharkhand)\
                     .filter(ee.Filter.eq('WRS_ROW',rowNumber2))\
                     .filter(ee.Filter.eq('WRS_ROW',rowNumber2))\
                     .filter(ee.Filter.eq('WRS_PATH',pathNumber))\
                     .first()
                
            #images = images_1.merge(images_2)
            images = ee.ImageCollection([images_1, images_2])
                
            

            # Check if images are found for the current month
            if images.size().getInfo() > 0:
                images_found = True

                # Take the mean of the images for the specified period
                image = images.mosaic()

                # Define output filename for Landsat
                landsat_filename = f'Landsat_{year}_{month:02d}.tif'
                landsat_output_path = os.path.join(output_folder, 'Landsat', landsat_filename)

                # Export each band of the Landsat image separately with export options
                for band_name, band_id in band_mapping[landsat_sensor].items():
                    band_image = image.select(band_id)

                    # Define output filename for the current band
                    band_filename = f'Landsat_{year}_{month:02d}_{band_name}.tif'
                    band_output_path = os.path.join(output_folder, 'Landsat', band_filename)

                    # Set export options for CRS and maxPixels
                    export_options = {
                        'crs': 'EPSG:32644',  # Change to your desired CRS
                        'maxPixels': 1e13  # Change to an appropriate value
                    }

                    # Export the current band to Google Drive with export_options
                    export_task = ee.batch.Export.image.toDrive(
                        image=band_image,
                        description=f'Landsat {band_name} Export',
                        folder=output_folder,
                        fileNamePrefix=f'Landsat/{band_filename}',
                        scale=30,
                        region=jharkhand,
                        **export_options  # Include export options
                    )
                    export_task.start()

                    # Wait for export task to complete
                    while export_task.active():
                        print(f'Exporting {band_name}...')
                        time.sleep(30)  # Wait for 30 seconds before checking again

                print(f'Export completed for Landsat {year}_{month:02d}')

                # Calculate NDVI, EVI, and NDWI
                band_info = band_mapping[landsat_sensor]
                ndvi = image.normalizedDifference([band_info['NIR'], band_info['RED']])
                evi = image.expression(
                    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                        'NIR': image.select(band_info['NIR']),
                        'RED': image.select(band_info['RED']),
                        'BLUE': image.select(band_info['BLUE'])
                    })
                ndwi = image.normalizedDifference([band_info['NIR'], band_info['SWIR']])

                # Define output filenames for NDVI, EVI, and NDWI
                ndvi_filename = f'Landsat_{year}_{month:02d}_NDVI.tif'
                evi_filename = f'Landsat_{year}_{month:02d}_EVI.tif'
                ndwi_filename = f'Landsat_{year}_{month:02d}_NDWI.tif'

                ndvi_output_path = os.path.join(output_folder, 'NDVI', ndvi_filename)
                evi_output_path = os.path.join(output_folder, 'EVI', evi_filename)
                ndwi_output_path = os.path.join(output_folder, 'NDWI', ndwi_filename)

                # Export NDVI, EVI, and NDWI to Google Drive with export options
                export_ndvi_task = ee.batch.Export.image.toDrive(
                    image=ndvi,
                    description='NDVI Export',
                    folder=output_folder,
                    fileNamePrefix='NDVI/' + ndvi_filename,
                    scale=30,
                    region=jharkhand,
                    **export_options  # Include export options
                )
                export_evi_task = ee.batch.Export.image.toDrive(
                    image=evi,
                    description='EVI Export',
                    folder=output_folder,
                    fileNamePrefix='EVI/' + evi_filename,
                    scale=30,
                    region=jharkhand,
                    **export_options  # Include export options
                )
                export_ndwi_task = ee.batch.Export.image.toDrive(
                    image=ndwi,
                    description='NDWI Export',
                    folder=output_folder,
                    fileNamePrefix='NDWI/' + ndwi_filename,
                    scale=30,
                    region=jharkhand,
                    **export_options  # Include export options
                )

                export_ndvi_task.start()
                export_evi_task.start()
                export_ndwi_task.start()

                # Wait for export tasks to complete
                while export_task.active() or export_ndvi_task.active() or export_evi_task.active() or export_ndwi_task.active():
                    print('Exporting...')  # You can replace this with a more informative message
                    time.sleep(30)  # Wait for 30 seconds before checking again

                print(f'Export completed for Landsat {year}_{month:02d}')

                break  # Exit the loop when an image is found for the current year

        if not images_found:
            print(f"No images found for year {year}. Moving to the next year.")

# Example usage:
start_year = 2004
end_year = 2014
feature_collection = ee.FeatureCollection("projects/webapp-385310/assets/studyarea_dl").geometry()
output_folder = 'GEE_Exports'  # Replace with your Google Drive folder name
cloudCover = 20
pathNumber = 136
rowNumber1 = 41
rowNumber2=42


# Export Landsat images for each year, mosaic them and export the mosaic
export_landsat_images(start_year, end_year, feature_collection, output_folder, cloudCover, pathNumber, rowNumber1,rowNumber2)


Exporting BLUE...


KeyboardInterrupt: 

In [10]:
import ee
import os
import time

def export_landsat_images(start_year, end_year, feature_collection, output_folder, cloudCover, pathNumber, rowNumber1,rowNumber2):
    ee.Initialize()

    # Define the region of interest
    jharkhand = feature_collection

    # Loop through the years
    for year in range(start_year, end_year + 1):
        images_found = False  # Flag to check if images are found for the current year

        # Create a band mapping dictionary based on the Landsat sensor
        band_mapping = {
            'LT05': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5','TIRS':'B6','GREEN':'B2'},
            'LE07': {'BLUE': 'B1', 'RED': 'B3', 'NIR': 'B4', 'SWIR': 'B5','TIRS':'B6_VCID_2','GREEN':'B2'},
            'LC08': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6','TIRS':'B10','GREEN':'B3'},
            'LC09': {'BLUE': 'B2', 'RED': 'B4', 'NIR': 'B5', 'SWIR': 'B6','TIRS':'B10','GREEN':'B3'}
        }

        # Get the Landsat sensor based on the year
        landsat_sensor = None
        if year < 2000:
            landsat_sensor = 'LT05'
        elif 2013 <= year < 2014:
            landsat_sensor = 'LE07'
        elif 2014 <= year < 2022:
            landsat_sensor = 'LC08'
        else:
            landsat_sensor = 'LC09'

        # Iterate through months from January to December
        for month in range(1, 13):
            # Set the start and end dates for the current month
            start_date = f'{year}-01-01'
            end_date = f'{year}-05-28'

            # Filter Landsat collection for the specified year and month
            landsat_collection = ee.ImageCollection(f"LANDSAT/{landsat_sensor}/C02/T1_TOA")

            images_1 = landsat_collection.filter(ee.Filter.lt('CLOUD_COVER', cloudCover)) \
                     .filter(ee.Filter.date(start_date, end_date)) \
                     .filterBounds(jharkhand)\
                     .filter(ee.Filter.eq('WRS_ROW',rowNumber1))\
                     .filter(ee.Filter.eq('WRS_ROW',rowNumber1))\
                     .filter(ee.Filter.eq('WRS_PATH',pathNumber))\
                     .mean()
            # Get the date of the first image in images_1
            #first_image_date = ee.Image(images_1).date()
            

            images_2 = landsat_collection.filter(ee.Filter.lt('CLOUD_COVER', cloudCover)) \
                     .filter(ee.Filter.date(start_date, end_date)) \
                     .filterBounds(jharkhand)\
                     .filter(ee.Filter.eq('WRS_ROW',rowNumber2))\
                     .filter(ee.Filter.eq('WRS_ROW',rowNumber2))\
                     .filter(ee.Filter.eq('WRS_PATH',pathNumber))\
                     .mean()
                
            #images = images_1.merge(images_2)
            images = ee.ImageCollection([images_1, images_2])
                
            

            # Check if images are found for the current month
            if images.size().getInfo() > 0:
                images_found = True

                # Take the mean of the images for the specified period
                image = images.mosaic()

                # Define output filename for Landsat
                landsat_filename = f'Landsat_{year}_{month:02d}.tif'
                landsat_output_path = os.path.join(output_folder, 'Landsat', landsat_filename)

                # Export each band of the Landsat image separately with export options
                for band_name, band_id in band_mapping[landsat_sensor].items():
                    band_image = image.select(band_id)

                    # Define output filename for the current band
                    band_filename = f'Landsat_{year}_{month:02d}_{band_name}.tif'
                    band_output_path = os.path.join(output_folder, 'Landsat', band_filename)

                    # Set export options for CRS and maxPixels
                    export_options = {
                        'crs': 'EPSG:32644',  # Change to your desired CRS
                        'maxPixels': 1e13  # Change to an appropriate value
                    }

                    # Export the current band to Google Drive with export_options
                    export_task = ee.batch.Export.image.toDrive(
                        image=band_image,
                        description=f'Landsat {band_name} Export',
                        folder=output_folder,
                        fileNamePrefix=f'Landsat/{band_filename}',
                        scale=30,
                        region=jharkhand,
                        **export_options  # Include export options
                    )
                    export_task.start()

                    # Wait for export task to complete
                    while export_task.active():
                        print(f'Exporting {band_name}...')
                        time.sleep(30)  # Wait for 30 seconds before checking again

                print(f'Export completed for Landsat {year}_{month:02d}')

                # Calculate NDVI, EVI, and NDWI
                band_info = band_mapping[landsat_sensor]
                ndvi = image.normalizedDifference([band_info['NIR'], band_info['RED']])
                evi = image.expression(
                    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                        'NIR': image.select(band_info['NIR']),
                        'RED': image.select(band_info['RED']),
                        'BLUE': image.select(band_info['BLUE'])
                    })
                ndwi = image.normalizedDifference([band_info['NIR'], band_info['SWIR']])

                # Define output filenames for NDVI, EVI, and NDWI
                ndvi_filename = f'Landsat_{year}_{month:02d}_NDVI.tif'
                evi_filename = f'Landsat_{year}_{month:02d}_EVI.tif'
                ndwi_filename = f'Landsat_{year}_{month:02d}_NDWI.tif'

                ndvi_output_path = os.path.join(output_folder, 'NDVI', ndvi_filename)
                evi_output_path = os.path.join(output_folder, 'EVI', evi_filename)
                ndwi_output_path = os.path.join(output_folder, 'NDWI', ndwi_filename)

                # Export NDVI, EVI, and NDWI to Google Drive with export options
                export_ndvi_task = ee.batch.Export.image.toDrive(
                    image=ndvi,
                    description='NDVI Export',
                    folder=output_folder,
                    fileNamePrefix='NDVI/' + ndvi_filename,
                    scale=30,
                    region=jharkhand,
                    **export_options  # Include export options
                )
                export_evi_task = ee.batch.Export.image.toDrive(
                    image=evi,
                    description='EVI Export',
                    folder=output_folder,
                    fileNamePrefix='EVI/' + evi_filename,
                    scale=30,
                    region=jharkhand,
                    **export_options  # Include export options
                )
                export_ndwi_task = ee.batch.Export.image.toDrive(
                    image=ndwi,
                    description='NDWI Export',
                    folder=output_folder,
                    fileNamePrefix='NDWI/' + ndwi_filename,
                    scale=30,
                    region=jharkhand,
                    **export_options  # Include export options
                )

                export_ndvi_task.start()
                export_evi_task.start()
                export_ndwi_task.start()

                # Wait for export tasks to complete
                while export_task.active() or export_ndvi_task.active() or export_evi_task.active() or export_ndwi_task.active():
                    print('Exporting...')  # You can replace this with a more informative message
                    time.sleep(30)  # Wait for 30 seconds before checking again

                print(f'Export completed for Landsat {year}_{month:02d}')

                break  # Exit the loop when an image is found for the current year

        if not images_found:
            print(f"No images found for year {year}. Moving to the next year.")

# Example usage:
start_year = 2004
end_year = 2014
feature_collection = ee.FeatureCollection("projects/webapp-385310/assets/studyarea_dl").geometry()
output_folder = 'GEE_Exports'  # Replace with your Google Drive folder name
cloudCover = 10
pathNumber = 136
rowNumber1 = 41
rowNumber2=42


# Export Landsat images for each year, mosaic them and export the mosaic
export_landsat_images(start_year, end_year, feature_collection, output_folder, cloudCover, pathNumber, rowNumber1,rowNumber2)


Exporting BLUE...
Exporting BLUE...
Exporting BLUE...
Exporting RED...
Exporting RED...
Exporting RED...
Exporting NIR...
Exporting NIR...
Exporting NIR...
Exporting NIR...
Exporting NIR...
Exporting SWIR...
Exporting SWIR...
Exporting SWIR...
Exporting TIRS...
Exporting TIRS...
Exporting TIRS...
Exporting TIRS...
Exporting TIRS...
Exporting GREEN...
Exporting GREEN...
Exporting GREEN...
Export completed for Landsat 2004_01
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Export completed for Landsat 2004_01
Exporting BLUE...
Exporting BLUE...
Exporting BLUE...
Exporting RED...
Exporting RED...
Exporting RED...
Exporting NIR...
Exporting NIR...
Exporting NIR...
Exporting SWIR...
Exporting SWIR...
Exporting SWIR...
Exporting TIRS...
Exporting TIRS...
Exporting TIRS...
Exporting GREEN...
Exporting GREEN...
Exporting GREEN...
Exporting GREEN...
Export completed for Landsat 2005_01
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Export complete